In [14]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np
import json

In [15]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15907, 19)

In [16]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    # end要加一天
    end = pd.to_datetime(end) + pd.Timedelta(days=1)
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

# japan_dead_fish

In [171]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [172]:
# 查找 2023.12.1 -> 2023.12.10之间，twitter上query为japan dead fish的
df_data_twitter = df_posts[df_posts['from'] == 'twitter']
df_data_twitter = get_post_date_range(df_data_twitter,'2023-12-01','2023-12-10')
df_data_twitter = df_data_twitter[df_data_twitter['query']== "Japan dead fish"]
df_data_twitter.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_twitter['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_twitter)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [173]:
df_data_weibo = df_posts[df_posts['from'] == 'weibo']
df_data_weibo = get_post_date_range(df_data_weibo,'2023-12-01','2023-12-10')
df_data_weibo = df_data_weibo[df_data_weibo['query'] == "w9"]
df_data_weibo.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_weibo['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_weibo)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [174]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [17]:
#

#

# election

In [183]:
cluster_dict = {
    1:('Great_Wave_Kanagawa', "2021-04-20", "2021-04-29"),
    2:('foreign_affairs_questions', '2021-04-20', '2021-04-29'),
    3:('japan_nuclear_wastewater', '2021-04-20', '2021-04-29'),
    4:('radioactive_condemn_water', '2021-04-20', '2021-04-29'),
    5:('240_china_nuclear_pollution', '2023-08-21', '2023-08-30'),
    6:('70_billion_japan_water', '2023-08-21', '2023-08-30'),
    7:('cooling_water_nuclear_wastewater', '2023-08-21', '2023-08-30'),
    8:('south_korea_nuclear_discharge', '2023-08-21', '2023-09-01'),
    9:('sue_TEPCO_japan', '2023-08-21', '2023-08-30'),
    10:('radioactive_pollution_japan_sea', '2023-08-21', '2023-08-30'),
    11:('treatment_japan_waste_nuclear', '2023-08-21', '2023-08-30'),
    12: ('japan_dead_fish', '2023-12-01', '2023-12-10'),
    13: ('border_...', '2023-12-21', '2024-03-25')
}


## border_...

In [18]:
df_posts.loc[df_posts['query'].isin([
    'shut down border', 'biden visits border', 'support of Texas', 'building a wall america', 'U.S.-Mexico border','US-Mexico border','trump visits border','e6','e9','']), 'cluster'] = "border_..."